# 💸 Projeto Final | Moneymap

### **Descrição do problema**
> Foi desenvolvido um sistema para controle financeiro que armazena as movimentações em arquivos nos formatos CSV ou JSON. O sistema permite a leitura, criação, atualização e a exclusão dos registros financeiros. Além disso, é capaz de gerar relatórios para calcular as receitas, despesas e os rendimentos dos investimentos.

### **Integrantes**:<a name="integrantes"></a>

>
- Beatriz Porto
- Eduardo Mattana
- Michel Camargo
- Pedro Lima
- Rubens Sousa

### Setup

In [ ]:
from datetime import datetime
import csv
import json
dados = {"1":"despesas","2":"receitas","3":"investimentos"}

### Funções auxiliares

In [ ]:
'''
  Função para escrever cada registro por meio do tipo do registro(Despesa,Receita,Investimento).

  input: lista de registros do tipo desejado.
  output: print dos valores dos registros linha por linha em formato tabular.

'''
def escreve_registro_tipo(elementos_tipo:list):
  for a in elementos_tipo:
        id = a["id_do_registro"]
        tipo_registro = a['tipo_de_registro']
        valor = a['valor_numerico']
        data = a['data_do_registro']
        print(f"{id: <5} {tipo_registro: <20} {valor: <10}  {data: <10}")

In [ ]:
'''
  Função para buscar o indice de um registro dentro de uma lista de registros.

  input: dicionário com registros, o tipo do registro e o id do registro.
  output: indice do registro.

'''
def busca_indice_registro(dicionario: dict, tipo_registro: str, id: int) -> int:
    lista_registros = dicionario.get(tipo_registro, [])
    cont = 0

    for registro in lista_registros:
        if registro['id_do_registro'] == id:
            return cont
        cont += 1


    return -1


In [ ]:
'''
  Função para calcular o montante de um investimento.

  input: capital, a variação de tempo em dias e uma taxa.
  output: o montante calculado.

'''
def calcula_montante(capital:float, variacao_dias:int,taxa:float)->float:
    return capital * (1 + taxa)**variacao_dias

In [ ]:
'''
  Função para alterar o tipo de um registro.

  input: registro a ser alterado e o novo tipo.
  output: registro com o tipo alterado.

'''
def atualiza_tipo_registro(registro:dict,novo_tipo:str)->dict:
  registro['tipo_de_registro'] = novo_tipo
  registro['data_do_registro'] = datetime.now().strftime("%d/%m/%y")
  return registro

In [ ]:
'''
  Função para alterar o valor de um registro.

  input: registro a ser alterado e o novo valor.
  output: registro com o valor alterado.

'''
def atualiza_valor_registro(registro:dict,novo_valor:float)->dict:
  registro['valor_numerico'] = novo_valor
  registro['data_do_registro'] = datetime.now().strftime("%d/%m/%y")
  return registro

In [ ]:
'''
  Função para permitir que apenas elementos que ainda não existam no dicionário
  atual sejam adicionados a ele.

  input: dicionário antigo a ser atualizado e novo dicionario com os novos
  registros para serem adicionados.
  output: validação completa.

'''
def atualiza_registro_repetido(dicionario_antigo:dict,dicionario_novo:dict)->None:
  for key, value in dicionario_novo.items():
    if key in dicionario_antigo:
        for novo_registro in value:
            if novo_registro not in dicionario_antigo[key]:
                dicionario_antigo[key].append(novo_registro)
    else:
      dicionario_antigo[key] = value

### Funções Principais

In [ ]:
'''
  Função para acrescentar , de forma manual, novos registros no dicionário de
  registros.

  input: dicionario desatualizado.
  output: dicionario atualizado.

'''
def adicionar_registro(dict_geral:dict) -> dict:

    while True:
        tipo = input("Tipo (Receita/Despesa/Investimento) ou 0 para sair: ").title()
        if tipo == '0':
            break
        if tipo not in ['Receita', 'Despesa', 'Investimento']:
            print("O tipo tem que ser Receita, Despesa, Investimento ou 0 para sair.")
            continue

        while True:
            try:
                valor = float(input("Valor: "))
                if tipo == 'Despesa' and valor <= 0:
                    print("O valor de despesas deve ser positivo.")
                else:
                    if tipo == 'Despesa':
                        valor = -valor
                    break
            except ValueError:
                print("Valor inválido.")

        if tipo=="Despesa":
            dict_geral["despesas"].append({'id_do_registro': len(dict_geral["despesas"])+1,
                                           'tipo_de_registro': tipo,
                                          'valor_numerico': valor,
                                           'data_do_registro': datetime.now().strftime("%d/%m/%y")})
        elif tipo=="Receita":
            dict_geral["receitas"].append({'id_do_registro': len(dict_geral["receitas"])+1,
                                           'tipo_de_registro': tipo,
                                           'valor_numerico': valor,
                                            'data_do_registro': datetime.now().strftime("%d/%m/%y")})

        else:
            dict_geral["investimentos"].append({'id_do_registro': len(dict_geral["investimentos"])+1,
                                                'tipo_de_registro': tipo,
                                                'valor_numerico': valor,
                                                'data_do_registro': datetime.now().strftime("%d/%m/%y")})


    return dict_geral

In [ ]:
'''
  Função para escrever todos os registros e seus  dados em formato tabular.

  input: dicionário com todos os registros.
  output: print de todos os registros em formato tabular.

'''

def escreve_registros(dict_geral:dict) -> None:

  print(f"{'ID':<5} {'Tipo de Registro':<20} {'Valor':<10} {'Data_registro': <10}")
  print("-" * 60)
  receitas = dict_geral.get("receitas")
  despesas= dict_geral.get("despesas")
  investimentos= dict_geral.get("investimentos")
  escreve_registro_tipo(receitas)
  escreve_registro_tipo(despesas)
  escreve_registro_tipo(investimentos)
  print("-" * 60)



In [ ]:

'''
  Função para criar o dicionário com os registros por meio da leitura de
  um arquivo csv.

  input: nome do arquivo a ser lido.
  output: dicionário atualizado.

'''
def coloca_registro_carga_csv(nome_arquivo:str)->dict:

  dicionario_registros = {}

  with open(nome_arquivo, 'r') as arquivo:

    planilha = list(csv.reader(arquivo, delimiter=',', lineterminator='\n'))

    cabecalho = planilha[0]

    dicionario_registros["despesas"] = [dict(zip(cabecalho,linha)) for linha in planilha if linha[0]=="Despesa"]
    dicionario_registros["receitas"] = [dict(zip(cabecalho,linha)) for linha in planilha if linha[0]=="Receita"]
    dicionario_registros["investimentos"] = [dict(zip(cabecalho,linha)) for linha in planilha if linha[0]=="Investimento"]


  return dicionario_registros


In [ ]:

'''
  Função para criar o dicionário com os registros por meio da leitura de
  um arquivo Json.

  input: nome do arquivo a ser lido.
  output: dicionário atualizado.

'''
def coloca_registro_carga_json(nome_arquivo:str)->dict:

  arquivo = open(nome_arquivo, 'r')

  with open(nome_arquivo, 'r') as arquivo:

    dicionario = json.load(arquivo)

  return dicionario




In [ ]:
'''
  Função para exportar um arquivo Json com os registros.

  input: dicionário com resgistros.
  output: arquivo Json.

'''
def exporta_registro_json(dicionario:dict) -> None:

  with open('dados.json', 'w') as arquivo:
    json.dump(dicionario,arquivo)


In [ ]:
'''
  Função para exportar um arquivo csv com os registros.

  input: dicionário com resgistros.
  output: arquivo csv.

'''

def exporta_registro_csv(dicionario:dict) -> None:

  colunas =["tipo_de_registro","data_do_registro","id_do_registro","valor_numerico"]
  registros_despesas = dicionario["despesas"]
  registros_investimentos = dicionario["investimentos"]
  registros_receitas = dicionario["receitas"]
  with open('dados.csv', 'w') as arquivo:

      escritor_csv = csv.DictWriter(arquivo, fieldnames=colunas)

      escritor_csv.writeheader()

      escritor_csv.writerows(registros_despesas)
      escritor_csv.writerows(registros_investimentos)
      escritor_csv.writerows(registros_receitas)

In [ ]:
'''
  Função atualizar o rendimento de um investimento especificado.

  input: dicionário com registros, id do investimento e taxa para cálculo
  do montante.
  output: dicionário com investimento atualizado.

'''

def atualizar_rendimento(dicionario:dict,id:int,taxa:float=0.0035)->dict:

  lista_investimentos = dicionario["investimentos"]
  investimento = {}
  for a in lista_investimentos:
    if a["id_do_registro"] == id:
      investimento = a

  capital = investimento['valor_numerico']
  data_investimento = datetime.strptime(investimento['data_do_registro'], "%d/%m/%y").date()
  data_atual = datetime.now().date()
  diferenca_data_dias = (data_atual-data_investimento).days
  indice = busca_indice_registro(dicionario,"investimentos",id)
  montante = calcula_montante(capital,diferenca_data_dias,taxa)
  investimento["valor_numerico"] = montante
  investimento['data_do_registro'] = data_atual.strftime("%d/%m/%y")
  dicionario["investimentos"][indice] = investimento


  return dicionario

In [ ]:
'''
  Função para deletar registro identificado pelo tipo e id.

  input:dicionário com registros, id do registro que se deseja excluir,
  tipo do registro.
  output: realização de exclusão de registro no dicionário.

'''
def deletar_registro(dicionario:dict,id:int,tipo_registro:str) -> None:
  dicionario[tipo_registro].pop(busca_indice_registro(dicionario,tipo_registro,id))


In [ ]:
'''
  Função para deletar registro identificado pelo tipo e id.

  input:dicionário com registros, id do registro que se deseja excluir,
  tipo do registro.
  output: realização de exclusão de registro no dicionário.

'''
def busca_registro(dicionario,tipo_registro,id) -> None:
  return dicionario[tipo_registro][busca_indice_registro(dicionario,tipo_registro,id)]

In [ ]:
'''
  Função para realizar alterações de tipo ou valor de um registro de um dicionário.

  input: dicionário com o registro a ser alterado.
  output: dicionário com as alterações realizadas.

'''

def atualiza_dicionario(dicionario:dict) -> None:
  print("Escolha entre as opções abaixo o tipo do registro que deseja atualizar: ")
  print("Digite 1 para alterar uma despesa")
  print("Digite 2 para alterar uma receita")
  print("Digite 3 para alterar um investimento")
  opcao_registro = int(input("Digite a opção desejada: "))
  id = int(input("Agora digite o id do registro que deseja alterar: "))
  print("Qual elemento do registro deseja alterar?")
  print("Digite 1 para alterar o tipo do  registro")
  print("Digite 2 para alterar o valor do registro")
  opcao_alteracao = int(input("Digite a opção desejada: "))
  opcao_novo_tipo = 0
  if opcao_alteracao==1:
    while True:
      print("Para qual novo tipo deseja alterar o registro?")
      print("Digite 1 para alterar para uma despesa")
      print("Digite 2 para alterar para uma receita")
      print("Digite 3 para alterar para um investimento")
      opcao_novo_tipo = int(input("Digite a opção desejada: "))

      if opcao_novo_tipo == opcao_registro:
        print("Opcao de alteração igual a opção original, escolha outra opção")
        continue
      else:
        break

    registro = dicionario[dados[str(opcao_registro)]][busca_indice_registro(dicionario,dados[str(opcao_registro)],id)]
    deletar_registro(dicionario,id,dados[str(opcao_registro)])

    if opcao_registro==1:
      registro["valor_numerico"] =  (registro["valor_numerico"])*-1

    if opcao_novo_tipo==1:
      registro["valor_numerico"] =  (registro["valor_numerico"])*-1

    novo_registro = atualiza_tipo_registro(registro,dados[str(opcao_novo_tipo)])
    dicionario[dados[str(opcao_novo_tipo)]].append(novo_registro)

  elif opcao_alteracao==2:

    while True:
      print("Qual o novo valor que deseja?")
      novo_valor = float(input("Digite um valor: "))
      if isinstance(novo_valor, float):
        break
      else:
        print("Valor digitado não é um número, tente novamente.")
        continue

    if opcao_registro==1 and novo_valor>0:
      novo_valor=novo_valor*-1

    if opcao_registro!=1 and novo_valor<0:
      novo_valor=novo_valor*-1

    registro = dicionario[dados[str(opcao_registro)]][busca_indice_registro(dicionario,dados[str(opcao_registro)],id)]
    registro = atualiza_valor_registro(registro,novo_valor)
    dicionario[dados[str(opcao_registro)]][busca_indice_registro(dicionario,dados[str(opcao_registro)],id)]=registro

In [ ]:
'''
  Função para retornar o somatório do valor numérico por
  tipo de registro por ano.

  input: dicionário com listas de registros, ano a ser analisado e o
  tipo de registro.
  output: somatório do valor numérico por tipo de registro.

'''
def retorna_soma_no_ano_tipo(dicionario:dict, ano:int, tipo:str)->float:
    registros = dicionario[tipo]

    def extrair_valor_se_corresponde(registro):
        data = datetime.strptime(registro['data_do_registro'], "%d/%m/%y")
        if data.year == ano:
            return registro['valor_numerico']
        return 0


    valores = map(extrair_valor_se_corresponde, registros)
    soma = sum(valores)

    return soma




In [ ]:
'''
  Função para retornar o somatório do valor numérico por
  tipo de registro por mês/ano.

  input: dicionário com listas de registros, mês a ser analisado,
  ano a ser analisado e o tipo de registro.
  output: somatório do valor numérico por tipo de registro por mês/ano.

'''
def retorna_soma_no_mes_tipo(dicionario:dict, mes:int,ano:int,tipo:str) -> float:
    registros = dicionario[tipo]

    def extrair_valor_se_corresponde(registro):
        data = datetime.strptime(registro['data_do_registro'], "%d/%m/%y")
        if data.month == mes and data.year == ano:
            return registro['valor_numerico']
        return 0


    valores = map(extrair_valor_se_corresponde, registros)
    soma = sum(valores)

    return soma

In [ ]:
'''
  Função para executar o menu do software.

'''

def menu_inicial():
  dicionario = {}
  dicionario["despesas"] = []
  dicionario["receitas"] = []
  dicionario["investimentos"] = []
  while True:
    print("---- Sistema de controle financeiro ----")
    print("1. Adicionar ou Atualizar registro de receita, despesa ou investimento;")
    print("2. Importar arquivo para leitura;")
    print("3. Mostrar todos os registros;")
    print("4. Buscar registro;")
    print("5. Excluir registro;")
    print("6. Exibir soma dos valores registrados;")
    print("7. Atualizar rendimentos;")
    print("8. Exportar arquivo;")
    print("0. para Sair;")

    opcao = int(input("Digite o número da opção desejada: "))
    if opcao == 1:
      opcao1 = int(input("Digite 1 para adicionar registros ou 2 para atualizar: "))
      if opcao1 == 1:
        dicionario = adicionar_registro(dicionario)
        print("Registros adicionados.")
      elif opcao1 == 2:
        atualiza_dicionario(dicionario)
        print("Registro atualizado")
    elif opcao == 2:
      opcao2=int(input("Digite 1 para importar arquivo Json ou 2 para CSV: "))
      if opcao2 == 1:
        dicionario_novo = coloca_registro_carga_csv("dados.json")
        atualiza_registro_repetido(dicionario,dicionario_novo)
        print("Carga completa e registros adicionados.")
      elif opcao2 == 2:
        dicionario_novo = coloca_registro_carga_csv("dados.csv")
        atualiza_registro_repetido(dicionario,dicionario_novo)
        print("Carga completa e registros adicionados.")
    elif opcao == 3:
      escreve_registros(dicionario)
    elif opcao == 4:
      print("Escolha uma opção de tipo de registro abaixo: ")
      print("1 para despesas")
      print("2 para receitas")
      print("3 para investimentos")
      opcao4 = int(input("Escolha a opcao do tipo desejado: "))
      id = int(input("Escreva o id do registro desejado: "))
      registro = busca_registro(dicionario,dados[str(opcao4)],id)
      print(f"Registro encontrado:")
      print(f"id:{registro['id_do_registro']}\n")
      print(f"Tipo do registro:{registro['tipo_de_registro']}\n")
      print(f"Valor:{registro['valor_numerico']}\n")
      print(f"Data:{registro['data_do_registro']}\n")


    elif opcao == 5:
      print("Escolha uma opção de tipo de registro abaixo: ")
      print("1 para despesas")
      print("2 para receitas")
      print("3 para investimentos")
      opcao5 = int(input("Escolha a opcao do tipo desejado: "))
      id = int(input("Escreva o id do registro desejado: "))
      deletar_registro(dicionario,id,dados[str(opcao5)])
      for a in dicionario[dados[str(opcao5)]]
      print("Registro deletado com sucesso.")
      for a in dicionario[dados[str(opcao5)]]:
          a["id_do_registro"]  -= 1
    elif opcao == 6:
      print("Escolha uma opção de tipo de registro abaixo: ")
      print("1 para despesas")
      print("2 para receitas")
      print("3 para investimentos")
      tipo = int(input("Escolha a opcao do tipo desejado: "))
      opcao6 = int(input("Digite 1 ter a soma por ano ou 2 para ter a soma por mes/ano: "))
      if opcao6 == 1:
        ano = int(input("Digite o ano no qual deseja a soma do tipo escolhido: "))
        print(f"O somatório no ano {ano} de {dados[str(tipo)]} é {retorna_soma_no_ano_tipo(dicionario, ano, dados[str(tipo)])}")
      if opcao6 == 2:
        ano = int(input("Digite o ano no qual deseja a soma do tipo escolhido: "))
        mes = int(input("Digite o mes no qual deseja a soma do tipo escolhido: "))
        print(f"O somatório no ano {ano} no mes {mes} de {dados[str(tipo)]} é {retorna_soma_no_mes_tipo(dicionario,mes,ano,dados[str(tipo)])}")

    elif opcao == 7:
      taxa = float(input("Qual a taxa que deseja para atualizar o investimento?"))
      id = int(input("Qual o id do investimento que deseja atualizar o rendimento?"))
      dicionario = atualizar_rendimento(dicionario,id,taxa)
      print("Atualização concluida com sucesso")

    elif opcao==8:
        opcao8 = int(input("Digite 1 para exportar no formato csv ou 2 para exportar no formato json: "))
        if opcao8==1:
          exporta_registro_csv(dicionario)
          print("Arquivo exportado.")
        elif opcao8==2:
          exporta_registro_json(dicionario)
          print("Arquivo exportado.")

    elif opcao==0:
      break
    else:
      print("Número incorreto. Digite novamente.\n")
      continue

### Execução do software

In [ ]:
menu_inicial()